In [1]:
import os
import re
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint

from src.query_db import QueryDb

In [2]:
%load_ext autoreload
%autoreload 2 

project_dir = os.path.dirname(os.getcwd())
print('project direcotry is ', project_dir)
if project_dir not in sys.path:
    sys.path.append(project_dir)
print(sys.path)

#
pdb_dir = '/home/yuan/data/pdb/rsync'
data_dir = './data/'
outputs_dir = '/home/yuan/output/pdb_abag'

project direcotry is  /home/yuan/bio
['/home/yuan/anaconda3/envs/openfold-env/lib/python310.zip', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/lib-dynload', '', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/site-packages', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/site-packages/openfold-2.2.0-py3.10-linux-x86_64.egg', '/home/yuan/bio']


### myosin

In [3]:
query = f"""
    select A.chain, A.compound_no, A.pdb_id, A.chain_type, A.fragment, A.compound_text, B.name, B.pdb_url
    from compound A, pdb B
    where A.pdb_id = B.pdb_id 
        AND A.pdb_id IN (
            select distinct pdb_id from compound
            where compound_text REGEXP "myosin")
        AND A.pdb_id NOT IN (
            select distinct pdb_id from compound
            where chain_type REGEXP "heavy|light|single|antigen"
        )
    order by A.pdb_id, compound_no;
"""
df = QueryDb().list_data(query, True)
print(df.shape)

outfile = os.path.join(data_dir, 'fragment_myosin.xlsx')
df.to_excel(outfile, index=False)

(1234, 8)


In [ ]:
# check distinct complex
CREATE TEMPORARY TABLE myosin AS
    select distinct A.pdb_id
    from compound A, pdb B
    where A.pdb_id = B.pdb_id 
        AND A.pdb_id IN (
            select distinct pdb_id from compound
            where compound_text REGEXP "myosin")
        AND A.pdb_id NOT IN (
            select distinct pdb_id from compound
            where chain_type REGEXP "heavy|light|single|antigen"
        );

#
UPDATE pdb
SET complex_type = 'myosin'
WHERE pdb_id IN (
    select pdb_id from myosin
);

### ferritin

In [ ]:
query = f"""
    select A.chain, A.compound_no, A.pdb_id, A.chain_type, A.fragment, A.compound_text, B.name, B.pdb_url
    from compound A, pdb B
    where A.pdb_id = B.pdb_id 
        AND A.pdb_id IN (
            select distinct pdb_id from compound
            where compound_text REGEXP "ferritin")
        AND A.pdb_id NOT IN (
            select distinct pdb_id from compound
            where chain_type REGEXP "heavy|light|single|antigen"
        )
    order by A.pdb_id, compound_no;
"""
df = QueryDb().list_data(query, True)
print(df.shape)

outfile = os.path.join(data_dir, 'fragment_ferritin.xlsx')
df.to_excel(outfile, index=False)

In [ ]:
# check distinct complex
CREATE TEMPORARY TABLE ferritin AS
    select distinct A.pdb_id
    from compound A, pdb B
    where A.pdb_id = B.pdb_id 
        AND A.pdb_id IN (
            select distinct pdb_id from compound
            where compound_text REGEXP "ferritin")
        AND A.pdb_id NOT IN (
            select distinct pdb_id from compound
            where chain_type REGEXP "heavy|light|single|antigen"
        );

#
UPDATE pdb
SET complex_type = 'ferritin'
WHERE pdb_id IN (
    select pdb_id from ferritin
);

### kinase

In [4]:
query = f"""
    select A.chain, A.compound_no, A.pdb_id, A.chain_type, A.fragment, A.compound_text, B.name, B.pdb_url
    from compound A, pdb B
    where A.pdb_id = B.pdb_id 
        AND A.pdb_id IN (
            select distinct pdb_id from compound
            where compound_text REGEXP "ec_number"
             AND compound_text REGEXP "kinase")
        AND A.pdb_id NOT IN (
            select distinct pdb_id from compound
            where chain_type REGEXP "heavy|light|single|antigen"
        )
    order by A.pdb_id, compound_no;
"""
df = QueryDb().list_data(query, True)
print(df.shape)

outfile = os.path.join(data_dir, 'fragment_kinase.xlsx')
df.to_excel(outfile, index=False)

(15560, 8)


In [ ]:
CREATE TEMPORARY TABLE kinase AS
    select distinct A.pdb_id
    from compound A, pdb B
    where A.pdb_id = B.pdb_id 
        AND A.pdb_id IN (
            select distinct pdb_id from compound
            where compound_text REGEXP "ec_number"
             AND compound_text REGEXP "kinase")
        AND A.pdb_id NOT IN (
            select distinct pdb_id from compound
            where chain_type REGEXP "heavy|light|single|antigen"
        );

#
UPDATE pdb
SET complex_type = 'kinase'
WHERE pdb_id IN (
    select pdb_id from kinase
);

### thrombin

In [10]:
query = f"""
    select A.chain, A.compound_no, A.pdb_id, A.chain_type, A.fragment, A.compound_text, B.name, B.pdb_url
    from compound A, pdb B
    where A.pdb_id = B.pdb_id 
        AND A.pdb_id IN (
            select distinct pdb_id from compound
            where compound_text REGEXP "thrombin"
                AND chain_type NOT REGEXP "heavy|light|single|antigen"
        )
    order by A.pdb_id, compound_no;
"""
df = QueryDb().list_data(query, True)
print(df.shape)

outfile = os.path.join(data_dir, 'fragment_thrombin.xlsx')
df.to_excel(outfile, index=False)

(1870, 8)


### enzyme

In [16]:
query = f"""
    select A.chain, A.compound_no, A.pdb_id, A.chain_type, A.fragment, A.compound_text, B.name, B.pdb_url
    from compound A, pdb B
    where A.pdb_id = B.pdb_id 
        AND A.pdb_id IN (
            select distinct pdb_id from compound
            where compound_text REGEXP "ec_number"
                AND chain_type NOT REGEXP "heavy|light|single|antigen"
                AND B.complex_type is NULL
        )
    order by A.pdb_id, compound_no;
"""
df = QueryDb().list_data(query, True)
print(df.shape)

outfile = os.path.join(data_dir, 'fragment_enzyme.xlsx')
df.to_excel(outfile, index=False)

(1831, 8)


## Major histocompatibility complex
MHC

In [28]:
query = f"""
    select A.chain, A.compound_no, A.pdb_id, A.chain_type, A.fragment, A.compound_text, B.name, B.pdb_url
    from compound A, pdb B
    where A.pdb_id = B.pdb_id 
        AND A.pdb_id IN (
            select distinct pdb_id from compound
            where (compound_text REGEXP "mhc|hla|microglobulin" or name REGEXP "mhc|hla|microglobulin")
                AND chain_type NOT REGEXP "heavy|light|single|antigen"
        )
    order by A.pdb_id, compound_no;
"""
df = QueryDb().list_data(query, True)

def fn(x):
    if re.findall(r'alpha|heavy', x):
        return 'alpha'
    elif re.findall(r'microglobulin|light|beta', x):
        return 'beta'
    elif re.findall(r'gamma|peptid', x):
        return 'peptide'
    elif re.findall(r'receptor', x):
        return 't-receptor'
    return None
df['chain_type'] = df['compound_text'].map(fn)

print(df.shape)

outfile = os.path.join(data_dir, 'MHC.xlsx')
df.to_excel(outfile, index=False)

(5232, 8)


In [30]:
# update compount.chain_type
infile = os.path.join(data_dir, 'label', 'MHC.xlsx')
xls_file = pd.ExcelFile(infile)
table = xls_file.parse('Sheet1')

In [35]:
# update table complex
from src.build_db import BuildDb

records = [(pdb_id, 'MHC') for pdb_id in table['pdb_id'].unique()]
BuildDb(records).insert_complex()

In [36]:
#  update table compound
QueryDb().update_compound(table, 'chain_type')
QueryDb().update_compound(table, 'fragment')

chain_type: succeed=5140, unchanged=21
fragment: succeed=252, unchanged=4909


### transcription factor

In [60]:
query = f"""
    select A.chain, A.compound_no, A.pdb_id, A.chain_type, A.fragment, A.compound_text, B.pdb_url
    from compound A, pdb B
    where A.pdb_id = B.pdb_id 
        AND A.pdb_id IN (
            select distinct pdb_id from compound
            where (compound_text REGEXP "transcription" and compound_text REGEXP "factor")
        )
    order by A.pdb_id, compound_no;
"""
df = QueryDb().list_data(query, True)
print(df.shape)

def fn(x):
    if re.findall(r'transcriptional factor', x):
        return 'tf'
    elif re.findall(r'dna ', x):
        return 'dna'        
    return 'factor'
df['chain_type'] = df['compound_text'].map(fn)

outfile = os.path.join(data_dir, 'tf.xlsx')
df.to_excel(outfile, index=False)

(11520, 7)


### G protein

In [66]:
query = f"""
    select A.chain, A.compound_no, A.pdb_id, A.chain_type, A.fragment, A.compound_text, B.pdb_url
    from compound A, pdb B
    where A.pdb_id = B.pdb_id 
        AND A.pdb_id IN (
            select distinct pdb_id from compound
            where (compound_text REGEXP " g protein|g binding protein|dopamine|adrenergic|opioid" 
                and compound_text REGEXP " g protein|g binding protein|dopamine|adrenergic|opioid")
        )
    order by A.pdb_id, compound_no;
"""
df = QueryDb().list_data(query, True)
print(df.shape)

def fn(x):
    if re.findall(r'g[ |-]protein', x):
        return 'gpro'
    elif re.findall(r'receptor ', x):
        return 'receptor'
    return 'factor'
df['chain_type'] = df['compound_text'].map(fn)

outfile = os.path.join(data_dir, 'gpro.txt')
df.to_csv(outfile, sep='\t', index=False)

outfile = os.path.join(data_dir, 'gpro.xlsx')
df.to_excel(outfile, index=False)

(1665, 7)


In [ ]:
### protease

In [69]:
query = f"""
    select A.chain, A.compound_no, A.pdb_id, A.chain_type, A.fragment, A.compound_text, B.pdb_url
    from compound A, pdb B
    where A.pdb_id = B.pdb_id 
        AND A.pdb_id IN (
            select distinct pdb_id from compound
            where (compound_text REGEXP "protease" and compound_text REGEXP "protease")
            AND (compound_text Not REGEXP "thrombin" and compound_text NOT REGEXP "thrombin")
        )
    order by A.pdb_id, compound_no;
"""
df = QueryDb().list_data(query, True)
print(df.shape)

def fn(x):
    if re.findall(r'protease', x):
        return 'protease'
    return 'factor'
df['chain_type'] = df['compound_text'].map(fn)

outfile = os.path.join(data_dir, 'protease.txt')
df.to_csv(outfile, sep='\t', index=False)

outfile = os.path.join(data_dir, 'protease.xlsx')
df.to_excel(outfile, index=False)

(11328, 7)


### ribosome

In [57]:
query = f"""
    select A.chain, A.compound_no, A.pdb_id, A.chain_type, NULL as weight, NULL as molecule, A.fragment, A.compound_text, B.pdb_url
    from compound A, pdb B
    where A.pdb_id = B.pdb_id 
        AND A.pdb_id IN (
            select distinct pdb_id from compound
            where (compound_text REGEXP "ribosom" or name REGEXP "ribosom")
        )
    order by A.pdb_id, compound_no;
"""
df = QueryDb().list_data(query, True)
df.shape

def fn(x):
    if re.findall(r'ec_number', x):
        return 'enzyme'
    elif re.findall(r'rrna|ribosomal rna', x):
        return 'rrna'
    elif re.findall(r'rna ', x):
        return 'rna'        
    elif re.findall(r'ribosomal protein', x):
        return 'rpro'
    return 'factor'
df['chain_type'] = df['compound_text'].map(fn)

def fn(x):
    pros= re.findall(r'\d+s', x)
    if pros:    
        return pros[0]
    return None
df['weight'] = df['compound_text'].map(fn)

def fn(x):
    pros= re.findall(r' [s|l|bl]\d+', x)
    if pros:    
        return pros[0]
    return None
df['molecule'] = df['compound_text'].map(fn)

outfile = os.path.join(data_dir, 'ribosome.xlsx')
df.to_excel(outfile, index=False)

In [ ]:
# update compount.chain_type
# infile = os.path.join(data_dir, 'label', 'ribosome.xlsx')
# xls_file = pd.ExcelFile(infile)
# table = xls_file.parse('Sheet1')

# QueryDb().update_compound(table, 'chain_type')
# QueryDb().update_compound(table, 'fragment')

In [45]:
query = f"""
    select A.chain, A.compound_no, A.pdb_id, A.chain_type, A.fragment, A.compound_text, B.name, B.pdb_url
    from compound A, pdb B
    where A.pdb_id = B.pdb_id
        AND A.pdb_id NOT IN (
            select pdb_id from compound
            where chain_type REGEXP 'heavy|light|single|antigen'
            union
            select pdb_id from complex where complex_type
        )
    order by A.pdb_id, compound_no;
"""
df = QueryDb().list_data(query, True)
print(df.shape)

outfile = os.path.join(data_dir, 'fragment_other.xlsx')
df.to_excel(outfile, index=False)

(356717, 8)
